# Posting List operations

This notebook is to evaluate inverted index and tf_idf on any given datasets.
For usage adapt the last 4 variables in the first cell (inverted_index_path, tf_idf_path, out_path and query_files) accordingly, depending on whether everything should be executed fo train or eval.

todos:
-   review everything
-   run for train and test set
-   check output file parameter if rank -> tf_idf is correct or this is meant elsewise --> Ciwan confirmed is fine, must just be sorted :)

In [19]:
import shelve
import pickle
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import pandas as pd
import numpy as np
import os
import math 
from collections import Counter
from collections import defaultdict 
from datetime import datetime
import re
import math 

nltk.download('stopwords')

try:
    import orjson as _json
    loads = _json.loads
except Exception:
    import json as _json
    loads = _json.loads

stop = set(stopwords.words('english') + list(string.punctuation))

INVERTED_INDEX_PATH = "./data/inverted_index_train.db" # "./data/inverted_index_eval.db"
TF_IDF_PATH = "./data/tf_idf_train.db"
OUT_PATH = "../data/runs/train"
DOC_LENGTH_PATH = "../data/doc_length_train"
query_files = ["../data/tot25/subsets/train80/train80-queries-dev1.jsonl", "../data/tot25/subsets/train80/train80-queries-dev2.jsonl", "../data/tot25/subsets/train80/train80-queries-dev3.jsonl", "../data/tot25/subsets/train80/train80-queries-train.jsonl"]
# query_files = ["../data/tot25/subsets/eval20/eval20-queries-dev1.jsonl", "../data/tot25/subsets/eval20/eval20-queries-dev2.jsonl", "../data/tot25/subsets/eval20/eval20-queries-dev3.jsonl", "../data/tot25/subsets/eval20/eval20-queries-train.jsonl"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
with shelve.open(DOC_LENGTH_PATH) as db_lengths:
    print(db_lengths[str(700)])

{'len': 7941}


Storing the doc_length in memory, allows for way faster lookup :)

In [4]:
with shelve.open(DOC_LENGTH_PATH) as db_length:
    doc_len_cache = {k: v["len"] for k, v in db_length.items()}

print(len(doc_len_cache))

100000


## Function for Lematization

In [5]:
# download for lematization
nltk.download('punkt_tab')   
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

def get_wordnet_pos(tag): # mapping POS tag from pos_tag to a format WordNetLemmatizer accepts.
    match tag[0]:
        case 'J':
            return wordnet.ADJ
        case 'V':
            return wordnet.VERB
        #case 'N':
        #    return wordnet.NOUN       online source: here, but twice no benefit so removed
        case 'R':
            return wordnet.ADV
        case _:
            return wordnet.NOUN

lemmatizer = WordNetLemmatizer()
def lematization(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    return [lemmatizer.lemmatize(token, get_wordnet_pos(tag)).lower() for token, tag in tagged] # lower so can compare eg write friday instead of Friday

#sanity check
lem = lematization("This notebook is used to create an inverted_index and tf_idf database for the provided corpus file (train or eval). Adapt the variables in the first cell accordingly for files and output dir.Note that the executino time for eval is 1/3 that of train, despite the fact that eval contains 2.5 times the amount of docs. We assume this is ")
print(lem) # later need to remove stopwords

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['this', 'notebook', 'be', 'use', 'to', 'create', 'an', 'inverted_index', 'and', 'tf_idf', 'database', 'for', 'the', 'provided', 'corpus', 'file', '(', 'train', 'or', 'eval', ')', '.', 'adapt', 'the', 'variable', 'in', 'the', 'first', 'cell', 'accordingly', 'for', 'file', 'and', 'output', 'dir.note', 'that', 'the', 'executino', 'time', 'for', 'eval', 'be', '1/3', 'that', 'of', 'train', ',', 'despite', 'the', 'fact', 'that', 'eval', 'contain', '2.5', 'time', 'the', 'amount', 'of', 'doc', '.', 'we', 'assume', 'this', 'be']


## Inverted Index

In [6]:
# Function to tokenize and preprocess a document
def preprocess_unique(text):
    #old function changed to lematization    tokens = set(word_tokenize(text.lower())) # get all tokens
    tokens = set(lematization(text))
    return [i for i in tokens if i not in stop] # get all tokens without stopwords

def AND(l1, l2):
    # note: this method assumes that both lists are sorted and do not contain any duplicates (which if the input comes from a posting list is given)
    # this asssumption is given considering the inverted index and tf_idf db's were created
    p1 = 0
    p2 = 0
    sqrtJump1 = max(math.isqrt(len(l1)), 1)
    sqrtJump2 = max(math.isqrt(len(l2)), 1)
    result = []
    while p1 < len(l1)  and p2 < len(l2):
        if l1[p1] == l2[p2]:
            result.append(l1[p1])
            p1 += 1
            p2 += 1
        # skip pointer in first list
        elif p1 % sqrtJump1 == 0 and p1 + sqrtJump1 < len(l1) and l1[p1 + sqrtJump1] <= l2[p2]:
            p1 += sqrtJump1
        # skip pointer in second list
        elif p2 % sqrtJump2 == 0 and p2 + sqrtJump2 < len(l2) and l2[p2 + sqrtJump2] <= l1[p1]:
            p2 += sqrtJump2
        elif l1[p1] < l2[p2]:
            p1 += 1
        else:
            p2 += 1
    return result

def OR(l1, l2):
    # note: this method assumes that both lists are sorted and do not contain any duplicates (which if the input comes from a posting list is given)
    # this asssumption is given considering the inverted index and tf_idf db's were created
    p1 = 0
    p2 = 0
    result = []
    while p1 < len(l1) and p2 < len(l2):
        if l1[p1] < l2[p2]:
            result.append(l1[p1])
            p1 += 1
        elif l1[p1] > l2[p2]:
            result.append(l2[p2])
            p2 += 1
        else: 
            result.append(l1[p1])
            p1 += 1
            p2 += 1
    while p1 < len(l1):
        result.append(l1[p1])
        p1 += 1
    while p2 < len(l2):
        result.append(l2[p2])
        p2 += 1
    return result

In [ ]:
def query_inverted_index_and(query):
    with shelve.open(INVERTED_INDEX_PATH) as db:
        terms = [t for t in preprocess_unique(query)]
        terms.sort(key = lambda t:len(db[t]))
        result = db[terms[0]]
        for term in terms[1:]:
            result = AND(result, db[term])
        return result

def query_inverted_index_sorted_by_word_counts_top_n(query, n):
    with shelve.open(INVERTED_INDEX_PATH) as db:
        terms = [t for t in preprocess_unique(query)]
        result = list()
        for term in terms:
            if not term in db:
                continue
            result.extend(db[term])
        return Counter(result).most_common(n)
    
def evaluate_inverted_index(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True) 
    for filename in query_files:
        print(filename)
        print(datetime.now())
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_inverted_index_sorted_by_word_counts_top_n(row["query"], 1000)
                rank_counter = 1
                for id, frequency in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {frequency} {runID}\n")
                    # print(f"{row.get("query_id")} Q0 {id} {rank_counter} {frequency} {runID}")
                    rank_counter += 1
                
                

evaluate_inverted_index(query_files, 1, OUT_PATH + "/inverted_index")
# 2 min eval, train 3:07 min

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 08:32:29.486744
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 08:32:51.636436
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 08:33:13.181600
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 08:35:17.240892


## Inverted Index with doc_length normalization

In [ ]:
def query_inverted_index_and(query):
    with shelve.open(INVERTED_INDEX_PATH) as db:
        terms = [t for t in preprocess_unique(query)]
        terms.sort(key = lambda t:len(db[t]))
        result = db[terms[0]]
        for term in terms[1:]:
            result = AND(result, db[term])
        return result

def query_inverted_index_sorted_by_word_counts_top_n(query, n):
    with shelve.open(INVERTED_INDEX_PATH) as db:
        with shelve.open(DOC_LENGTH_PATH) as db_lengths:
            terms = [t for t in preprocess_unique(query)]
            result = list()
            for term in terms:
                if not term in db:
                    continue
                result.extend(db[term])
            
            counter = Counter(result)
            for doc_id in counter:
                if str(doc_id) not in doc_len_cache:
                    counter[doc_id] = -1
                    print(f"error with {doc_id}")
                counter[doc_id] = counter[doc_id] / math.log(doc_len_cache[str(doc_id)] + 2)
            return counter.most_common(n)
    
def evaluate_inverted_index(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True) 
    for filename in query_files:
        print(filename)
        print(datetime.now())
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_inverted_index_sorted_by_word_counts_top_n(row["query"], 1000)
                rank_counter = 1
                for id, frequency in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {frequency} {runID}\n")
                    # print(f"{row.get("query_id")} Q0 {id} {rank_counter} {frequency} {runID}") 
                    rank_counter += 1
                
                

evaluate_inverted_index(query_files, 1, OUT_PATH + "/inverted_index_normalized_doc_length")
# eval: 5:56, train: 4:05

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 08:35:37.201332
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 08:36:05.535705
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 08:36:35.320988
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 08:39:14.092247


## TF_IDF

In [ ]:
def preprocess_unique_list(text):
    #old function changed to lematization    tokens = list(word_tokenize(text.lower())) # get all tokens
    tokens = list(lematization(text)) # get all tokens
    return [i for i in tokens if i not in stop] # get all tokens without stopwords


def query_tf_idf_top_n(query, n):
    with shelve.open(TF_IDF_PATH) as db:
        terms = [t for t in preprocess_unique_list(query)]
        
        tf_idf_sums = defaultdict(float)
        
        for term in terms:
            if term not in db:
                continue
            data = db[term]
            idf = data["idf"]
            doc_ids = data["doc_ids"]
            tfs = data["tfs"]

            # in the database the tfs for each document is stored for the documents index in doc_ids, for this reason we can easily zip them together :)
            for doc_id, tf in zip(doc_ids, tfs):
                tf_idf_sums[doc_id] +=  math.log(tf + 1) * idf
        
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]

def evaluate_tf_idf(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(row["query"], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1
            
evaluate_tf_idf(query_files, 1, OUT_PATH + "/tf_idf")
# 12:07 mins eval, 11:13 train

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 08:39:43.032899
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 08:40:31.636023
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 08:41:25.593501
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 08:48:39.697743


## TF_IDF with doc_length normalization

In [ ]:
def preprocess_unique_list(text):
    #old function changed to lematization    tokens = list(word_tokenize(text.lower())) # get all tokens
    tokens = list(lematization(text)) # get all tokens
    return [i for i in tokens if i not in stop] # get all tokens without stopwords


def query_tf_idf_top_n(query, n):
    with shelve.open(TF_IDF_PATH) as db:
            terms = [t for t in preprocess_unique_list(query)]
            
            tf_idf_sums = defaultdict(float)
            
            for term in terms:
                if term not in db:
                    continue
                data = db[term]
                idf = data["idf"]
                doc_ids = data["doc_ids"]
                tfs = data["tfs"]

                # in the database the tfs for each document is stored for the documents index in doc_ids, for this reason we can easily zip them together :)
                for doc_id, tf in zip(doc_ids, tfs):
                    tf_idf_sums[doc_id] +=  math.log(tf + 1) * idf
                
            for doc_id in tf_idf_sums:
                if str(doc_id) not in doc_len_cache:
                    print(f"error with {doc_id}")
                    tf_idf_sums[doc_id] = -1
                    continue
                tf_idf_sums[doc_id] /= math.log(doc_len_cache[str(doc_id)] + 2)
                
            return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]

def evaluate_tf_idf(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(row["query"], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1
            
evaluate_tf_idf(query_files, 1, OUT_PATH + "/tf_idf_normalized_doc_length")
# 9min eval, 13 mins train

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 08:50:56.246144
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 08:52:38.722771
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 08:54:01.753001
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:02:37.868192


# Reduced queries without weights

In [11]:
def preprocess_unique_list(text):
    #old function changed to lematization    tokens = list(word_tokenize(text.lower())) # get all tokens
    tokens = list(lematization(text)) # get all tokens
    return [i for i in tokens if i not in stop] # get all tokens without stopwords

tokens = list(lematization("I am singing"))
print(tokens)

['i', 'be', 'sing']


In [ ]:
def query_tf_idf_top_n(query, n):
    with shelve.open(TF_IDF_PATH) as db:
        terms = [t for t in preprocess_unique_list(query)]
        
        tf_idf_sums = defaultdict(float)
        
        for term in terms:
            if term not in db:
                continue
            data = db[term]
            idf = data["idf"]
            doc_ids = data["doc_ids"]
            tfs = data["tfs"]

            # in the database the tfs for each document is stored for the documents index in doc_ids, for this reason we can easily zip them together :)
            for doc_id, tf in zip(doc_ids, tfs):
                tf_idf_sums[doc_id] +=  math.log(tf + 1) * idf
        
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]

def evaluate_tf_idf_reduced_query(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        filename_improved_query = "./data/improved_queries/" + re.search(r'([^/]+)(?=\.[^.]+$)', filename).group(1) + "_improved_queries.jsonl"
        df_improved_queries = pd.read_json(filename_improved_query, lines = True)
        df_improved_queries["terms_string"] = df_improved_queries["keywords"].apply(lambda items: " ".join([d["term"] for d in items]))
        
        
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(df_improved_queries[df_improved_queries["id"] == row.get("query_id")]["terms_string"].iloc[0], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1
            
evaluate_tf_idf_reduced_query(query_files, 1, OUT_PATH + "/tf_idf_reduced_queries")
# 42 seconds eval, 2:37 min

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 09:03:54.453318
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 09:04:22.322902
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 09:04:48.668541
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:06:08.365338


## Reduced queries without Weigths with normalization for ducument length

In [ ]:
def query_tf_idf_top_n(query, n):
    with shelve.open(TF_IDF_PATH) as db:
        terms = [t for t in preprocess_unique_list(query)]
        
        tf_idf_sums = defaultdict(float)
        
        for term in terms:
            if term not in db:
                continue
            data = db[term]
            idf = data["idf"]
            doc_ids = data["doc_ids"]
            tfs = data["tfs"]

            # in the database the tfs for each document is stored for the documents index in doc_ids, for this reason we can easily zip them together :)
            for doc_id, tf in zip(doc_ids, tfs):
                tf_idf_sums[doc_id] +=  math.log(tf + 1) * idf
                
        for doc_id in tf_idf_sums:
            if str(doc_id) not in doc_len_cache:
                print(f"error with {doc_id}")
                tf_idf_sums[doc_id] = -1
                continue
            tf_idf_sums[doc_id] /= math.log(doc_len_cache[str(doc_id)] + 2)
                
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]

def evaluate_tf_idf_reduced_query(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        filename_improved_query = "./data/improved_queries/" + re.search(r'([^/]+)(?=\.[^.]+$)', filename).group(1) + "_improved_queries.jsonl"
        df_improved_queries = pd.read_json(filename_improved_query, lines = True)
        df_improved_queries["terms_string"] = df_improved_queries["keywords"].apply(lambda items: " ".join([d["term"] for d in items]))
        
        
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(df_improved_queries[df_improved_queries["id"] == row.get("query_id")]["terms_string"].iloc[0], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1
            
evaluate_tf_idf_reduced_query(query_files, 1, OUT_PATH + "/tf_idf_reduced_queries_normalized_doc_length")
# eval: 53 s, 4:33 train

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 09:06:32.020414
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 09:07:03.025346
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 09:07:31.407428
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:10:10.470272


# Distance Weighted retrieval

In this section we use TF-IDF but in addition weight the query terms by their index, i.e. first word gets weight 1, last gets minimum weight. 
The weighting function is 
1 - (1 - MIN_WEIGHT) * (index/ query_length)^2 

We chose this function for the following reasons:
1. Weights should be in the range of 1 max and MIN_WEIGHT = 0.4 min (minimum value worked the best with experience)
2. The function should be concave and decreasing slowly, hence the ^2
3. The weight should not only depend on the index, but also on the length of the query, i.e. first Token gets weight 1, last gets weight MIN_WEIGHT, no matter how long the query is. That is because our query lengths range from tens to hundreds.

In [ ]:
MIN_WEIGHT = 0.4 
def query_tf_idf_top_n(query, n):
    with shelve.open(TF_IDF_PATH) as db:
        terms = [t for t in preprocess_unique_list(query)]
        term_count = len(terms)
        
        tf_idf_sums = defaultdict(float)
        
        counter = 0
        
        for term in terms:
            if term not in db:
                continue
            data = db[term]
            idf = data["idf"]
            doc_ids = data["doc_ids"]
            tfs = data["tfs"]

            # in the database the tfs for each document is stored for the documents index in doc_ids, for this reason we can easily zip them together :)
            for doc_id, tf in zip(doc_ids, tfs):
                tf_idf_sums[doc_id] +=  math.log(1 + tf) * idf * (1 - (1 - MIN_WEIGHT) * ((counter / term_count) ** 2))
            counter += 1
        
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]

def evaluate_tf_idf_reduced_query(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        filename_improved_query = "./data/improved_queries/" + re.search(r'([^/]+)(?=\.[^.]+$)', filename).group(1) + "_improved_queries.jsonl"
        df_improved_queries = pd.read_json(filename_improved_query, lines = True)
        df_improved_queries["terms_string"] = df_improved_queries["keywords"].apply(lambda items: " ".join([d["term"] for d in items]))
        
        
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(df_improved_queries[df_improved_queries["id"] == row.get("query_id")]["terms_string"].iloc[0], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1
            
evaluate_tf_idf_reduced_query(query_files, 1, OUT_PATH + "/tf_idf_distance_weighted")
# 39 sec eval, 3:58 train

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 09:11:05.080458
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 09:11:53.973594
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 09:12:25.456504
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:14:26.086336


# Distance Weighted retrieval normalized by document length

In [ ]:
MIN_WEIGHT = 0.4 
def query_tf_idf_top_n(query, n):
    with shelve.open(TF_IDF_PATH) as db:
        terms = [t for t in preprocess_unique_list(query)]
        term_count = len(terms)
        
        tf_idf_sums = defaultdict(float)
        
        counter = 0
        
        for term in terms:
            if term not in db:
                continue
            data = db[term]
            idf = data["idf"]
            doc_ids = data["doc_ids"]
            tfs = data["tfs"]

            # in the database the tfs for each document is stored for the documents index in doc_ids, for this reason we can easily zip them together :)
            for doc_id, tf in zip(doc_ids, tfs):
                tf_idf_sums[doc_id] +=  math.log(1 + tf) * idf * (1 - (1 - MIN_WEIGHT) * ((counter / term_count) ** 2))
            counter += 1
        
        for doc_id in tf_idf_sums:
            if str(doc_id) not in doc_len_cache:
                print(f"error with {doc_id}")
                tf_idf_sums[doc_id] = -1
                continue
            tf_idf_sums[doc_id] /= math.log(doc_len_cache[str(doc_id)] + 2)
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]

def evaluate_tf_idf_reduced_query(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        filename_improved_query = "./data/improved_queries/" + re.search(r'([^/]+)(?=\.[^.]+$)', filename).group(1) + "_improved_queries.jsonl"
        df_improved_queries = pd.read_json(filename_improved_query, lines = True)
        df_improved_queries["terms_string"] = df_improved_queries["keywords"].apply(lambda items: " ".join([d["term"] for d in items]))
        
        
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(df_improved_queries[df_improved_queries["id"] == row.get("query_id")]["terms_string"].iloc[0], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1
            
evaluate_tf_idf_reduced_query(query_files, 1, OUT_PATH + "/tf_idf_distance_weighted_normalized_doc_length")
# eval: 1:08, train: 4:23 

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 09:15:03.602240
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 09:15:53.683718
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 09:16:32.482135
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:18:46.401190


# Reduced queries weights

No we also account for the weights given by ChatGPT. For each proposed phrase, like "action thriller", we preprocess it and split it into unique terms, then compute the min tf over all terms and the maximum corresponding idf over all unique terms in the phrase, based on that we compute a weighted sum tf-idf over all proposed phrases.
1. minimum tf: since we do not store phrases in our tf-idf, but only unique tokens, we make the assumption that the given terms in a phrase co-occur in the amount of the minimum tf  of the terms. We are aware that this might not be the case, however due to computational constraints and for storage reasons we cannot compute the tf-idf based on phrases, which is why we make this assumption.
2. maximum idf: Since in (1) we assume that the occurence of the phrase is defined by the least occuring term in the phrase, we take the maximum idf, as this corresponds to the least occuring term.

In [ ]:
def query_tf_idf_top_n(terms_weights, n):
    with shelve.open(TF_IDF_PATH) as db:
        tf_idf_sums = defaultdict(float) # default value is 0 :)
        for entry in terms_weights:
            datas = []
            # db does not contain subterm --> ignore it for quering
            preprocessed_split = preprocess_unique_list(entry["term"])
            if any(term not in db for term in preprocessed_split):
                continue
            for term in preprocessed_split:
                datas.append(db[term])
            if len(datas) == 0:
                continue
            idf = max(data["idf"] for data in datas)
            doc_ids = set.intersection(*(set(data["doc_ids"]) for data in datas))
            min_tfs = defaultdict(lambda: float("inf"))
            for data in datas:
                for doc_id, tf in zip(data["doc_ids"], data["tfs"]):
                    min_tfs[doc_id] = min(min_tfs[doc_id], tf)
            for doc_id in doc_ids:
                tf_idf_sums[doc_id] += math.log(min_tfs[doc_id] + 1) *  idf * entry["weight"]
                
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]
   

def evaluate_tf_idf_reduced_query(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        filename_improved_query = "./data/improved_queries/" + re.search(r'([^/]+)(?=\.[^.]+$)', filename).group(1) + "_improved_queries.jsonl"
        df_improved_queries = pd.read_json(filename_improved_query, lines = True)
        
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(df_improved_queries[df_improved_queries["id"] == row.get("query_id")].iloc[0]["keywords"], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1

            
evaluate_tf_idf_reduced_query(query_files, 1, OUT_PATH + "/tf_idf_reduced_queries_weights")
# eval: 42 s, 2:47 train

../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 09:19:26.949701
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 09:19:58.695464
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 09:20:25.926935
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:21:49.832909


# Reduced queries weights normalized document length

In [ ]:
def query_tf_idf_top_n(terms_weights, n):
    with shelve.open(TF_IDF_PATH) as db:
        tf_idf_sums = defaultdict(float) # default value is 0 :)
        for entry in terms_weights:
            datas = []
            # db does not contain subterm --> ignore it for quering
            preprocessed_split = preprocess_unique_list(entry["term"])
            if any(term not in db for term in preprocessed_split):
                continue
            for term in preprocessed_split:
                datas.append(db[term])
            if len(datas) == 0:
                continue
            idf = max(data["idf"] for data in datas)
            doc_ids = set.intersection(*(set(data["doc_ids"]) for data in datas))
            min_tfs = defaultdict(lambda: float("inf"))
            for data in datas:
                for doc_id, tf in zip(data["doc_ids"], data["tfs"]):
                    min_tfs[doc_id] = min(min_tfs[doc_id], tf)
            for doc_id in doc_ids:
                tf_idf_sums[doc_id] += math.log(min_tfs[doc_id] + 1) *  idf * entry["weight"]
        
        
        for doc_id in tf_idf_sums:
            if str(doc_id) not in doc_len_cache:
                print(f"error with {doc_id}")
                tf_idf_sums[doc_id] = -1
                continue
            tf_idf_sums[doc_id] /= math.log(doc_len_cache[str(doc_id)] + 2)    
        return sorted(tf_idf_sums.items(), key = lambda x: x[1], reverse = True)[:n]
   

def evaluate_tf_idf_reduced_query(query_files, runID, output_path):
    os.makedirs(output_path, exist_ok=True)
    for filename in query_files:
        print(filename)
        print(datetime.now())
        filename_improved_query = "./data/improved_queries/" + re.search(r'([^/]+)(?=\.[^.]+$)', filename).group(1) + "_improved_queries.jsonl"
        df_improved_queries = pd.read_json(filename_improved_query, lines = True)
        
        with open(output_path + "/" + filename.split("/")[-1].split(".")[0] + ".run", "w") as output_file:
            testdata = pd.read_json(filename, lines = True)
            for index, row in testdata.iterrows():
                result = query_tf_idf_top_n(df_improved_queries[df_improved_queries["id"] == row.get("query_id")].iloc[0]["keywords"], 1000)
                rank_counter = 1
                for id, tf_idf in result:
                    output_file.write(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}\n")
                    # print(f"{row.get('query_id')} Q0 {id} {rank_counter} {tf_idf} {runID}")
                    rank_counter += 1

            
evaluate_tf_idf_reduced_query(query_files, 1, OUT_PATH + "/tf_idf_reduced_queries_weights_normalized_doc_length")
# eval: 40s, 2:38 train


../data/tot25/subsets/train80/train80-queries-dev1.jsonl
2025-11-27 09:22:14.742339
../data/tot25/subsets/train80/train80-queries-dev2.jsonl
2025-11-27 09:22:43.262804
../data/tot25/subsets/train80/train80-queries-dev3.jsonl
2025-11-27 09:23:08.545726
../data/tot25/subsets/train80/train80-queries-train.jsonl
2025-11-27 09:24:28.285669


# Evaluation of above created run files

In [18]:
from pathlib import Path
import subprocess

print(OUT_PATH)
root_path = Path(OUT_PATH)

for root, dirs, files in os.walk(root_path):
    print("Directory:", root)
    for d in dirs:
        print("  Subdirectory:", os.path.join(root, d))
    for f in files:
        result = subprocess.run(f"python ../EvalPipelineSubSet/run_eval.py --split {f.split(".")[0].split("-")[-1]} --run {os.path.join(root, f)} --metrics ndcg@10 ndcg@1000 R@1000", shell=True, capture_output=True, text=True)
        print("  File:", f, "\t|", result.stdout )
        print("  File:", f)
        print("    returncode:", result.returncode)
        print("    stdout:", result.stdout)
        print("    stderr:", result.stderr)

../data/runs/train
Directory: ..\data\runs\train
  Subdirectory: ..\data\runs\train\inverted_index
  Subdirectory: ..\data\runs\train\inverted_index_normalized_doc_length
  Subdirectory: ..\data\runs\train\tf_idf
  Subdirectory: ..\data\runs\train\tf_idf_distance_weighted
  Subdirectory: ..\data\runs\train\tf_idf_distance_weighted_normalized_doc_length
  Subdirectory: ..\data\runs\train\tf_idf_normalized_doc_length
  Subdirectory: ..\data\runs\train\tf_idf_reduced_queries
  Subdirectory: ..\data\runs\train\tf_idf_reduced_queries_normalized_doc_length
  Subdirectory: ..\data\runs\train\tf_idf_reduced_queries_weights
  Subdirectory: ..\data\runs\train\tf_idf_reduced_queries_weights_normalized_doc_length
Directory: ..\data\runs\train\inverted_index
  File: train80-queries-dev1.run 	| [Warning] 29 queries in qrels but not in run. Examples: ['152', '403', '813', '1011', '898']
Explanation: missing queries are scored as 0 and still averaged; this is standard TREC practice
Since the metric va

# Results train
| method                                          | split | NDCG@10 | NDCG@1000 | R@1000 |
|------------------------------------------------|-------|---------|-----------|--------|
| inverted_index                                  | dev1  | 0.000   | 0.001     | 0.007 |
| inverted_index                                  | dev2  | 0.000   | 0.002     | 0.014 |
| inverted_index                                  | dev3  | 0.001   | 0.020     | 0.149 |
| inverted_index                                  | train | 0.000   | 0.000     | 0.000 |
| inverted_index_normalized_doc_length           | dev1  | 0.007   | 0.016     | 0.077 |
| inverted_index_normalized_doc_length           | dev2  | 0.002   | 0.012     | 0.077 |
| inverted_index_normalized_doc_length           | dev3  | 0.026   | 0.058     | 0.274 |
| inverted_index_normalized_doc_length           | train | 0.000   | 0.004     | 0.028 |
| tf_idf                                         | dev1  | 0.000   | 0.005     | 0.042 |
| tf_idf                                         | dev2  | 0.003   | 0.006     | 0.035 |
| tf_idf                                         | dev3  | 0.001   | 0.030     | 0.205 |
| tf_idf                                         | train | 0.000   | 0.001     | 0.007 |
| tf_idf_distance_weighted                       | dev1  | 0.000   | 0.025     | 0.197 |
| tf_idf_distance_weighted                       | dev2  | 0.007   | 0.034     | 0.217 |
| tf_idf_distance_weighted                       | dev3  | 0.357   | 0.389     | 0.629 |
| tf_idf_distance_weighted                       | train | 0.002   | 0.021     | 0.168 |
| tf_idf_distance_weighted_normalized_doc_length | dev1  | 0.006   | 0.049     | 0.303 |
| tf_idf_distance_weighted_normalized_doc_length | dev2  | 0.025   | 0.067     | 0.343 |
| tf_idf_distance_weighted_normalized_doc_length | dev3  | 0.466   | 0.485     | 0.655 |
| tf_idf_distance_weighted_normalized_doc_length | train | 0.007   | 0.045     | 0.287 |
| tf_idf_normalized_doc_length                   | dev1  | 0.000   | 0.012     | 0.092 |
| tf_idf_normalized_doc_length                   | dev2  | 0.007   | 0.019     | 0.112 |
| tf_idf_normalized_doc_length                   | dev3  | 0.013   | 0.051     | 0.291 |
| tf_idf_normalized_doc_length                   | train | 0.000   | 0.005     | 0.042 |
| tf_idf_reduced_queries                         | dev1  | 0.000   | 0.026     | 0.204 |
| tf_idf_reduced_queries                         | dev2  | 0.007   | 0.032     | 0.196 |
| tf_idf_reduced_queries                         | dev3  | 0.361   | 0.391     | 0.625 |
| tf_idf_reduced_queries                         | train | 0.002   | 0.020     | 0.154 |
| tf_idf_reduced_queries_normalized_doc_length   | dev1  | 0.010   | 0.050     | 0.303 |
| tf_idf_reduced_queries_normalized_doc_length   | dev2  | 0.021   | 0.067     | 0.364 |
| tf_idf_reduced_queries_normalized_doc_length   | dev3  | 0.471   | 0.490     | 0.655 |
| tf_idf_reduced_queries_normalized_doc_length   | train | 0.009   | 0.045     | 0.280 |
| tf_idf_reduced_queries_weights                 | dev1  | 0.000   | 0.033     | 0.225 |
| tf_idf_reduced_queries_weights                 | dev2  | 0.010   | 0.042     | 0.266 |
| tf_idf_reduced_queries_weights                 | dev3  | 0.412   | 0.435     | 0.634 |
| tf_idf_reduced_queries_weights                 | train | 0.004   | 0.026     | 0.182 |
| tf_idf_reduced_queries_weights_normalized_doc_length | dev1 | 0.019 | 0.054 | 0.275 |
| tf_idf_reduced_queries_weights_normalized_doc_length | dev2 | 0.016 | 0.058 | 0.315 |
| tf_idf_reduced_queries_weights_normalized_doc_length | dev3 | 0.471 | 0.491 | 0.646 |
| tf_idf_reduced_queries_weights_normalized_doc_length | train | 0.008 | 0.039 | 0.252 |


# Results eval
| method                                          | split  | NDCG@10 | NDCG@1000 | R@1000 |
|------------------------------------------------|--------|---------|-----------|--------|
| inverted_index                                  | dev1  | 0.000   | 0.002     | 0.014 |
| inverted_index                                  | dev2  | 0.000   | 0.002     | 0.014 |
| inverted_index                                  | dev3  | 0.003   | 0.009     | 0.050 |
| inverted_index                                  | train | 0.000   | 0.002     | 0.014 |
| inverted_index_normalized_doc_length           | dev1  | 0.007   | 0.009     | 0.021 |
| inverted_index_normalized_doc_length           | dev2  | 0.007   | 0.012     | 0.042 |
| inverted_index_normalized_doc_length           | dev3  | 0.014   | 0.024     | 0.088 |
| inverted_index_normalized_doc_length           | train | 0.000   | 0.003     | 0.028 |
| tf_idf                                         | dev1  | 0.000   | 0.001     | 0.007 |
| tf_idf                                         | dev2  | 0.000   | 0.004     | 0.035 |
| tf_idf                                         | dev3  | 0.002   | 0.010     | 0.056 |
| tf_idf                                         | train | 0.000   | 0.003     | 0.021 |
| tf_idf_normalized_doc_length                   | dev1  | 0.000   | 0.003     | 0.021 |
| tf_idf_normalized_doc_length                   | dev2  | 0.000   | 0.006     | 0.049 |
| tf_idf_normalized_doc_length                   | dev3  | 0.007   | 0.017     | 0.078 |
| tf_idf_normalized_doc_length                   | train | 0.000   | 0.005     | 0.042 |
| tf_idf_distance_weighted                       | dev1  | 0.000   | 0.006     | 0.049 |
| tf_idf_distance_weighted                       | dev2  | 0.002   | 0.018     | 0.119 |
| tf_idf_distance_weighted                       | dev3  | 0.083   | 0.094     | 0.170 |
| tf_idf_distance_weighted                       | train | 0.000   | 0.011     | 0.070 |
| tf_idf_distance_weighted_normalized_doc_length | dev1  | 0.004   | 0.017     | 0.113 |
| tf_idf_distance_weighted_normalized_doc_length | dev2  | 0.020   | 0.038     | 0.140 |
| tf_idf_distance_weighted_normalized_doc_length | dev3  | 0.118   | 0.126     | 0.183 |
| tf_idf_distance_weighted_normalized_doc_length | train | 0.010   | 0.021     | 0.105 |
| tf_idf_reduced_queries                         | dev1  | 0.000   | 0.005     | 0.042 |
| tf_idf_reduced_queries                         | dev2  | 0.003   | 0.019     | 0.119 |
| tf_idf_reduced_queries                         | dev3  | 0.081   | 0.093     | 0.172 |
| tf_idf_reduced_queries                         | train | 0.002   | 0.011     | 0.070 |
| tf_idf_reduced_queries_normalized_doc_length   | dev1  | 0.007   | 0.021     | 0.127 |
| tf_idf_reduced_queries_normalized_doc_length   | dev2  | 0.022   | 0.039     | 0.140 |
| tf_idf_reduced_queries_normalized_doc_length   | dev3  | 0.116   | 0.124     | 0.181 |
| tf_idf_reduced_queries_normalized_doc_length   | train | 0.007   | 0.020     | 0.098 |
| tf_idf_reduced_queries_weights                 | dev1  | 0.000   | 0.009     | 0.070 |
| tf_idf_reduced_queries_weights                 | dev2  | 0.012   | 0.027     | 0.119 |
| tf_idf_reduced_queries_weights                 | dev3  | 0.094   | 0.102     | 0.170 |
| tf_idf_reduced_queries_weights                 | train | 0.000   | 0.007     | 0.049 |
| tf_idf_reduced_queries_weights_normalized_doc_length | dev1 | 0.002 | 0.016 | 0.113 |
| tf_idf_reduced_queries_weights_normalized_doc_length | dev2 | 0.030 | 0.045 | 0.133 |
| tf_idf_reduced_queries_weights_normalized_doc_length | dev3 | 0.113 | 0.120 | 0.175 |
| tf_idf_reduced_queries_weights_normalized_doc_length | train | 0.000 | 0.011 | 0.077 |